# Lab 4.4 CNN Using VGG-16

In this exercise we will borrow the pre-trained weights and convolutional layers from the famous VGG16 network to our model. VGG16 has been trained on some millions of images of daily objects. The weights of such should be well tuned against various features and abstract representation of the objects. They will be useful for recognizing images of plants and seedlings as well.

This technique is sometimes referred as "Transferred Learning" in some readings.

---

To start this lab 4.4, remember to restart the runtime of your Colab to avoid running out of resources in the runtime.

## Import necessary packages

Let's import some packages first. The first few cells should be familiar to you.

In [0]:
#====================================================#
# This python notebook is created by Oscar PANG (oscarpang@vtc.edu.hk),
# Assistant Project Officer of VTC STEM Education Centre
# For distribution, please quote the author
#====================================================#

import os, sys
import cv2
import time
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import pyplot
from glob import glob

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

!unzip /content/gdrive/My\ Drive/public/plant*.zip 

## Upload dataset

The dataset should be in the VM, as long as you have not reset the runtime or the 12-hour usage period is not yet lapsed. If not, load it again using the codes in the previous exercise.

In [0]:
# Run the following command to browse the "plant_seedlings_dataset" directory. 
# You should see two folders "test" and "train"
ls plant_seedlings_dataset

## Define constants

The input dimension required by VGG-16 is 224 x 224. Hence we have to resize the image data in order to fit the input requirement.

In [0]:
# define some constants and path names
label_path = "plant_seedlings_dataset/train"
train_path = "plant_seedlings_dataset/train/*/*.png"
test_path = "plant_seedlings_dataset/test/*.png"

LENGTH = 224
IMG_SIZE = (LENGTH, LENGTH)
CHANNEL = 3

## Prepare and load the dataset

First let's load the filenames of the plant seedling images and get the class names of the labels.



In [0]:
# use glob to read the path of each image file
train_filenames = glob(train_path)
test_filenames = glob(test_path)

#print(train_filenames)
#print(test_filenames)

label_dict = {}
class_num = 0

# create a dictionary for the key-value pairs of seed names and the respective values
for subdir in sorted(os.listdir(label_path)):
    label_dict[subdir] = class_num
    class_num+=1

print(label_dict)

Next, we randomize the order of the filename first.

In [0]:
train_filenames = np.asarray(train_filenames)

np.random.seed(5)
indices = np.random.permutation(train_filenames.shape[0])

train_filenames = train_filenames[indices]

train_num = int(train_filenames.shape[0] * 0.8)

Then, we read the image data and the labels into the lists according to the randomized order of the filenames. This may take a while.

In [0]:
x_Train = []
y_Train = []
x_Eval = []
y_Eval = []

cnt = 0

for filename in train_filenames:
    image = cv2.resize(cv2.imread(filename), IMG_SIZE)
    image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    seed_name = filename.split("/")[2]
    if cnt < train_num:
      x_Train.append(image)
      y_Train.append(label_dict[seed_name])
    else:
      x_Eval.append(image)
      y_Eval.append(label_dict[seed_name])
    cnt += 1
    

# convert the lists into numpy array
x_Train = np.asarray(x_Train)
y_Train = np.asarray(y_Train)
x_Eval = np.asarray(x_Eval)
y_Eval = np.asarray(y_Eval)

# examine the datasets
print(x_Train.shape)
print(y_Train.shape)
print(x_Eval.shape)
print(y_Eval.shape)

We can visualize the image data read from the folders. (You can skip this part)

In [0]:
plt.rcParams['figure.figsize'] = (10.0,8.0)
for i in range(20):
    plt.subplot(4,5,i+1)
    plt.imshow(x_Train[i])


## Data Processing

We don't do rescaling using numpy here as in the previous exercise. We will rescale it using ImageDataGenerator of Keras in the training runtime. This will save some resources in the VM.

In [0]:
from keras.utils import np_utils

# perform one-hot encoding to the train labels
y_Train = np_utils.to_categorical(y_Train)
y_Eval = np_utils.to_categorical(y_Eval)

print(y_Train[:20])
print(y_Eval[:20])

## Define helper functions

In [0]:
# create helper function
def show_train_history(train_history, train, validation):
    plt.plot(train_history.history[train])
    plt.plot(train_history.history[validation])
    plt.title('Train History')
    plt.xlabel('Epochs')
    plt.ylabel(train)
    plt.show()

## Build CNN Model

In [0]:
# network building..
# import packages
from keras import backend as K
from keras.models import Model
from keras.layers import Flatten, Dense, Dropout
from keras.optimizers import Adam, RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
from keras.callbacks import TensorBoard

Next, import the pre-trained weights and architecture from VGG16 model. We freeze the weights of the convolution layers of the model and import it into our model.

In [0]:
vgg_conv = VGG16(weights='imagenet', include_top=False, input_shape=(LENGTH, LENGTH, 3))

# Freeze the layers except the last 4 layers
for layer in vgg_conv.layers[:-4]:
    layer.trainable = False

# Check the trainable status of the individual layers
for layer in vgg_conv.layers:
    print(layer, layer.trainable)

In [0]:
vgg_conv.summary()

We buld our CNN on top of the VGG-16 convolution layers.

In [0]:
from keras.models import Sequential

# Create the model
model = Sequential()
 
# Add the vgg convolutional base model
model.add(vgg_conv)

model.add(Flatten())
model.add(Dense(256, activation = 'relu'))
model.add(Dropout(0.25))
model.add(Dense(512, activation = 'relu'))
model.add(Dropout(0.25))
#model.add(Dense(256, activation = 'relu'))
#model.add(Dropout(0.25))
model.add(Dense(12, activation = 'softmax'))

model.summary()

model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=1e-4), metrics=['accuracy'])


We perform rescaling of image data (0-255 to 0-1) using `ImageDataGenerator` of keras at runtime. This reduces memory consumption in the VM before training.

In [0]:
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)


We use a smaller batch size (32) because of huge resource demand in VGG. Also, we use fit_generator for flowing train data and label from the `ImageDataGenerator`.

We begin training with 10 epochs first. We can increase the epochs later.

In [0]:
start_time = time.time()
#train_history = model.fit(x_Train, y_Train, epochs=40, batch_size=128, validation_data=(x_Eval, y_Eval))


train_history = model.fit_generator(train_datagen.flow(x_Train, y_Train, batch_size=32),
                    steps_per_epoch=len(x_Train) / 32, epochs=10, 
                    validation_data=test_datagen.flow(x_Eval, y_Eval, batch_size=32),
                    validation_steps=20)
end_time = time.time()

In [0]:
print("Total time elapsed = %d sec" % int(end_time - start_time))
print("="*70)
print()
# evaluate model accuracy
scores, accuracy = model.evaluate(x_Eval, y_Eval, batch_size=32)
print("model scores = ", scores)
print("model accuracy = ", accuracy)

show_train_history(train_history, 'acc', 'val_acc')
#show_train_history(train_history, 'loss', 'val_loss')

With a simple CNN having 3 convolution layers, you have built a model achieving 70% of accuracy. This is not bad though! 

But this is not the end! We should strive for a better result. Now save the model for later use and move on to the next part of Lab 4!

## Save Trained Model

In [0]:
weight_file = str(time.strftime("%Y%m%d_%H%M%S"))
weight_file = 'seedling_classifier_VGG_' + weight_file + '.h5'
model.save(weight_file)
print("Model weights have been saved successfully!")

In [0]:
# serialize model to JSON
model_json = model.to_json()
model_name = 'model_vgg16_'+str(time.strftime("%Y%m%d_%H%M%S"))+'.json'
with open(model_name, "w") as json_file:
    json_file.write(model_json)

print("Saved model architecture to disk")

In [0]:
!ls

# Download the saved model to your local system
from google.colab import files
files.download(filename)
files.download(model_name)